In [11]:
# 處理Json資料

import requests
import pandas as pd
import json

url = "https://www.twse.com.tw/rwd/zh/fund/BFI82U?dayDate=20241018&response=json"
res = requests.get(url)
jd = json.loads(res.text)
df = pd.DataFrame(jd['data'], columns = jd['fields'])
df

,單位名稱,買進金額,賣出金額,買賣差額
0,自營商(自行買賣),"9,264,701,664","4,313,902,149","4,950,799,515"
1,自營商(避險),"18,259,746,454","13,551,024,927","4,708,721,527"
2,投信,"5,508,558,763","6,162,329,649","-653,770,886"
3,外資及陸資(不含外資自營商),"200,630,321,684","144,009,367,275","56,620,954,409"
4,外資自營商,0,0,0
5,合計,"233,663,328,565","168,036,624,000","65,626,704,565"


In [16]:
# 處理HTML資料

import requests
import pandas as pd

url = "https://www.twse.com.tw/rwd/zh/fund/BFI82U?dayDate=20241018&response=html"
res = requests.get(url)
df = pd.read_html(StringIO(res.text), header = 1)[0] #指定標題列
df

,單位名稱,買進金額,賣出金額,買賣差額
0,自營商(自行買賣),9264701664,4313902149,4950799515
1,自營商(避險),18259746454,13551024927,4708721527
2,投信,5508558763,6162329649,-653770886
3,外資及陸資(不含外資自營商),200630321684,144009367275,56620954409
4,外資自營商,0,0,0
5,合計,233663328565,168036624000,65626704565


In [17]:
# 處理csv資料

import requests
import pandas as pd

url = "https://www.twse.com.tw/rwd/zh/fund/BFI82U?dayDate=20241018&response=csv"
res = requests.get(url)
df = pd.read_csv(StringIO(res.text), header = 1) #指定標題列
df.iloc[:6, :4]

,單位名稱,買進金額,賣出金額,買賣差額
0,自營商(自行買賣),"9,264,701,664","4,313,902,149","4,950,799,515"
1,自營商(避險),"18,259,746,454","13,551,024,927","4,708,721,527"
2,投信,"5,508,558,763","6,162,329,649","-653,770,886"
3,外資及陸資(不含外資自營商),"200,630,321,684","144,009,367,275","56,620,954,409"
4,外資自營商,0,0,0
5,合計,"233,663,328,565","168,036,624,000","65,626,704,565"


In [ ]:
# 下載三大法人歷史資料

import requests
import pandas as pd
import json
import datetime as dt
from dateutil.relativedelta import relativedelta
import time
import sqlite3 as lite #資料庫套件

#倒數計時

def countdown(t):
    while t > -1:
        min, sec = divmod(t, 60)
        if t > 0:
            print("{:02d}:{:02d}".format(min, sec), end='\r')  # 正確格式化 min 和 sec
        elif t == 0:
            print("{:02d}:{:02d}".format(min, sec), end='\n')  # 倒數結束時換行
        
        t -= 1
        time.sleep(1)

url = "https://www.twse.com.tw/rwd/zh/fund/BFI82U"
payload = {
    'dayDate' : '',
    'response' : 'json'
}
data = []
#加入日期
base = dt.datetime.today() - relativedelta(years = 0, months = 0, days = 15)  #設定從今天往前抓取的日期範圍
while base < dt.datetime.today():
    payload['dayDate'] = base.strftime("%Y%m%d")
    print(payload['dayDate']) #列印資料進度
    res = requests.get(url, params = payload)
    jd = json.loads(res.text)
    if jd['stat' ] == 'OK': #當交易日stat資料 ＝OK
        df = pd.DataFrame(jd['data'], columns = jd['fields'])
        df.insert(0, "日期", jd['date']) #數字 0 表示將新欄位插入為第一個欄位。
        df = df.iloc[:5, :4] #取前五行，前四列
        df = df.apply(lambda x: x.str.replace(',','')) #把逗號取代掉
        data.append(df)
        countdown(15) #每一次抓取後停留15秒
        
    base += relativedelta(days = 1) #以天為單位進行累加
df = pd.concat(data, ignore_index = True)

with lite.connect("每日行情.db") as con:
    df.to_sql("三大法人統計金額表", con = con, if_exists = 'replace') #如果有存在就覆蓋
df

In [33]:
#倒數計時

import time
t = 5

while t > -1:
    if t > 0:
        print("{:02d}".format(t), end = '\r')
    elif t == 0:
        print("{:02d}".format(t), end = '\n')
    
    t-= 1
    time.sleep(1)

00


In [41]:
#倒數計時秒數換成分鐘

import time
t = 65

while t > -1:
    min, sec = divmod(t, 60)
    if t > 0:
        print("{:02d}:{:02d}".format(min, sec), end='\r')  # 正確格式化 min 和 sec
    elif t == 0:
        print("{:02d}:{:02d}".format(min, sec), end='\n')  # 倒數結束時換行
    
    t -= 1
    time.sleep(1)


KeyboardInterrupt: 